In [1]:
import json
import numpy as np
import pandas as pd
import requests
import numpy
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

# Research Question: Find the top 10 players with the highest ratio between completed passes and attempted passes.

### Import the players dataset

In [2]:

players=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\players.json")
players
#Displaying maximum number
pd.set_option('display.max_columns', 14)
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null


### Import the event data for England

In [3]:
events=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\events\\events_England.json")
pd.set_option('display.max_columns', 15)

#### As we have to calculate the efficiency of the players based on the passes, we filter out the information related to passes. Thus we target the column 'eventName' and choose only those rows which contains information related to passes. 

In [4]:
events['eventName']=events['eventName'].astype(str)
events=events[(events.eventName.str.contains('Pass'))]
pd.set_option('display.max_columns', 15)

#### Splitting the column 'Tags' that is a dictionary

In [5]:
TAG1=[]
for i in range(len(events.tags)):
    try:
        TAG1.append(str(events.tags[i][0]['id']))
    except:
        TAG1.append('Notag')
TAG2=[]
for i in range(len(events.tags)):
    try:
        TAG2.append(str(events.tags[i][1]['id']))
    except:
        TAG2.append('Notag')
TAG3=[]
for i in range(len(events.tags)):
    try:
        TAG3.append(str(events.tags[i][2]['id']))
    except:
        TAG3.append('Notag')
TAG4=[]
for i in range(len(events.tags)):
    try:
        TAG4.append(str(events.tags[i][2]['id']))
    except:
        TAG4.append('Notag')
TAG1=pd.DataFrame({'Tag1':TAG1})
TAG2=pd.DataFrame({'Tag2':TAG2})
TAG3=pd.DataFrame({'Tag3':TAG3})
TAG4=pd.DataFrame({'Tag4':TAG4})
TAG=[TAG1,TAG2,TAG3,TAG4]
TAG=pd.concat(TAG ,axis=1, sort=False)
mix_events=[events,TAG]
events=pd.concat(mix_events,  axis=1,  sort=False).reindex(events.index)
events=events.drop('tags',axis=1)

In [6]:
events['eventName'] = events['eventName'].astype(str)
events=events[(events.eventName.str.contains('Pass'))]
events.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,25413.0,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719.0,Pass,1609.0,1H,2.758649,85,177959171.0,1801,Notag,Notag,Notag
1,8.0,High pass,370224.0,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719.0,Pass,1609.0,1H,4.946850,83,177959172.0,1801,Notag,Notag,Notag
2,8.0,Head pass,3319.0,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719.0,Pass,1609.0,1H,6.542188,82,177959173.0,1801,Notag,Notag,Notag
3,8.0,Head pass,120339.0,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719.0,Pass,1609.0,1H,8.143395,82,177959174.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,167145.0,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719.0,Pass,1609.0,1H,10.302366,85,177959175.0,1801,Notag,Notag,Notag


#### Counting the number of passes with respect to the player id

In [7]:
passes=events
passes=passes.drop('positions',axis=1)
passes.playerId=list(passes.playerId)
PLAYERS=[]
for i in range(len(passes.playerId)):
    PLAYERS.append(passes.playerId.iloc[i])
passes_player=pd.DataFrame(PLAYERS , columns=['PlayerId'])
passes_player['Numberofpasses'] = passes_player.groupby('PlayerId')['PlayerId'].transform('count')
passes_player.head()

,PlayerId,Numberofpasses
0,25413.0,657
1,370224.0,574
2,3319.0,1790
3,120339.0,834
4,167145.0,1877


#### In order to set the threshold of the number of passes, we calculate the first quartile of the number of passes. Then we set the lower limit as (1st quartile*1.5). The reason behind this is, while dealing with a set of variables, the upper and lower boundary is being set as  (3rd quartile*1.5) and (1st quartile*1.5) respectively. Here we are concerned about the players who have played very limited number of matches or insufficient amount of hours. That is why we set the lower limit as the only threshold.

In [8]:
#calculating 1st quartile
#quartile_1st= passes_player.Numberofpasses.quantile(0.25)
#quartile_3rd= passes_player.Numberofpasses.quantile(0.75)
#quartile_1st=int(quartile_1st)
#quartile_3rd=int(quartile_3rd)
#IQR=quartile_3rd-quartile_1st
#calculating lower bound
Mean=passes_player.Numberofpasses.mean()
std=passes_player.Numberofpasses.std()
Mean=int(Mean)
std=int(std)
lower_bound=Mean-std
ID_passes_threshold=[]
frequency_passes_threshold=[]
frequency_passes=[]
ID_passes=[]
for i in range(len(passes_player.Numberofpasses)):
    frequency_passes.append(float(passes_player.Numberofpasses.iloc[i]))
    ID_passes.append(float(passes_player.PlayerId.iloc[i]))
for i in range(len(frequency_passes)):
    if frequency_passes[i]>lower_bound:
        frequency_passes_threshold.append(int(frequency_passes[i]))
        ID_passes_threshold.append(int(ID_passes[i]))

In [9]:
passes_threshold=pd.DataFrame(ID_passes_threshold, columns=['PlayerId'])
passes_threshold.insert(1, "Numberofpasses",frequency_passes_threshold, True)
passes_threshold.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
passes_threshold.head()

,PlayerId,Numberofpasses
0,25413,657
1,370224,574
2,3319,1790
3,120339,834
4,167145,1877


In [10]:
events_tag2=events.loc[events['Tag2'] == '1801']
events_tag1=events.loc[events['Tag1']=='1801']
events_tag3=events.loc[events['Tag3']=='1801']
events_tag4=events.loc[events['Tag4']=='1801']
events_tag=[events_tag1,events_tag2,events_tag3,events_tag4]
success=pd.concat(events_tag, axis=0)
success.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,25413.0,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719.0,Pass,1609.0,1H,2.758649,85,177959171.0,1801,Notag,Notag,Notag
1,8.0,High pass,370224.0,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719.0,Pass,1609.0,1H,4.946850,83,177959172.0,1801,Notag,Notag,Notag
2,8.0,Head pass,3319.0,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719.0,Pass,1609.0,1H,6.542188,82,177959173.0,1801,Notag,Notag,Notag
3,8.0,Head pass,120339.0,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719.0,Pass,1609.0,1H,8.143395,82,177959174.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,167145.0,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719.0,Pass,1609.0,1H,10.302366,85,177959175.0,1801,Notag,Notag,Notag


In [11]:
SUCCESS=[]
for i in range(len(success.playerId)):
    SUCCESS.append(success.playerId.iloc[i])
success_new=pd.DataFrame(SUCCESS , columns=['PlayerId'])
success_new['Numberofpasses'] = success_new.groupby('PlayerId')['PlayerId'].transform('count')
success_new.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
success_new.head()

,PlayerId,Numberofpasses
0,25413.0,298
1,370224.0,185
2,3319.0,1012
3,120339.0,205
4,167145.0,937


In [12]:
final_passes=pd.merge(success_new, passes_threshold, on='PlayerId')
final_passes=final_passes.rename(columns={'Numberofpasses_x':'AccuratePassesNumber','Numberofpasses_y':'TotalPassesNumber'})
final_passes.head()

,PlayerId,AccuratePassesNumber,TotalPassesNumber
0,25413.0,298,657
1,370224.0,185,574
2,3319.0,1012,1790
3,120339.0,205,834
4,167145.0,937,1877


In [13]:
def make_col(df, col):
    return pd.concat([df.drop([col], axis=1), df[col].apply(pd.Series)], axis=1)
teams = pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\teams.json").rename(columns={'name':'teamname'})
teams = make_col(teams, "area")
teams = teams.drop(columns='city')
teams = teams.rename(columns={'name':'nation'})
engteams = teams[(teams["alpha3code"]=="XEN") & (teams["type"]=="club") | (teams["teamname"]=="Swansea City")]
engplayer = players.merge(engteams, left_on="currentTeamId", right_on="wyId")
engplayer=engplayer.drop(columns=['birthArea','height','role','currentNationalTeamId','officialName','type','middleName','lastName','passportArea','weight','currentTeamId','firstName','foot'],axis=1)

engplayer=engplayer.drop(columns=['alpha3code','alpha2code','id','nation','wyId_y','teamname'])
engplayer=engplayer.rename(columns={"wyId_x": "PlayerId"})

final_player_passes=pd.merge(engplayer, final_passes, on='PlayerId')
final_player_passes.head()

,birthDate,PlayerId,shortName,AccuratePassesNumber,TotalPassesNumber
0,1989-03-02,36,T. Alderweireld,598,898
1,1987-04-24,48,J. Vertonghen,1129,2370
2,1992-02-14,54,C. Eriksen,869,2196
3,1993-07-28,8717,H. Kane,246,594
4,1990-09-19,8945,K. Trippier,601,1311


In [14]:
Efficiency=[]
for i in range(len(final_player_passes.AccuratePassesNumber)):
    total=final_player_passes.TotalPassesNumber.iloc[i]
    accurate=final_player_passes.AccuratePassesNumber.iloc[i]
    Ratio=accurate/total
    Efficiency.append(Ratio)
Efficiency=pd.DataFrame(Efficiency)

In [15]:
final_player_passes.insert(4,'Efficiency',Efficiency)
final_player_ratio=final_player_passes
final_player_ratio.head()

,birthDate,PlayerId,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1989-03-02,36,T. Alderweireld,598,0.665924,898
1,1987-04-24,48,J. Vertonghen,1129,0.476371,2370
2,1992-02-14,54,C. Eriksen,869,0.395719,2196
3,1993-07-28,8717,H. Kane,246,0.414141,594
4,1990-09-19,8945,K. Trippier,601,0.458429,1311


In [16]:
final_player_ratio.to_json(r'C:\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw2\final_player_ratio.json')

In [17]:
bestefficiency=final_player_ratio.nlargest(10, ['Efficiency']) 
bestefficiency=bestefficiency.reset_index()
bestefficiency=bestefficiency.drop(columns=['index','PlayerId'])
bestefficiency=bestefficiency.set_value(9, 'shortName', 'V.Lindelof')

bestefficiency

C:\Users\Debodeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,birthDate,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1996-02-02,H. Winks,568,0.904459,628
1,1987-02-14,S. Dann,450,0.828729,543
2,1989-08-12,T. Cleverley,830,0.767808,1081
3,1988-07-03,W. Reid,490,0.757342,647
4,1990-01-05,L. Fer,435,0.741056,587
5,1986-08-20,A. Surman,814,0.734657,1108
6,1992-07-05,Alberto Moreno,584,0.730914,799
7,1986-02-26,Nacho Monreal,1200,0.708383,1694
8,1985-01-01,S. Davis,711,0.691634,1028
9,1992-12-23,V.Lindelof,460,0.690691,666


### The above table shows the top 10 most efficient players. We can observe that the highest efficiency is 76.78% in the premier league dataset.